In [1]:
# 1. Imports
import pandas as pd
import numpy as np
import re
from collections import Counter
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split

In [2]:
# 2. Load the Datasets
# Load the Kaggle dataset
kaggle_df = pd.read_csv('Data/Data.csv')

# Load the ISOT datasets
true_df = pd.read_csv('Data/True.csv')
fake_df = pd.read_csv('Data/Fake.csv')

# For ISOT, assign labels: assume true news = 1 and fake news = 0
true_df['label'] = 1
fake_df['label'] = 0

In [3]:
# 3. Combine the Datasets
# Combine the two ISOT datasets
isot_df = pd.concat([true_df, fake_df], ignore_index=True)

# Optionally, inspect shapes
print("Kaggle dataset shape:", kaggle_df.shape)
print("ISOT dataset shape:", isot_df.shape)

# Combine Kaggle and ISOT data into one DataFrame
df = pd.concat([kaggle_df, isot_df], ignore_index=True)

# Shuffle the combined dataset
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
print("Combined dataset shape:", df.shape)

Kaggle dataset shape: (20800, 5)
ISOT dataset shape: (44898, 5)
Combined dataset shape: (65698, 7)


In [4]:
# 4. Handle Missing Values
# Drop rows missing 'text' (critical for classification)
df = df.dropna(subset=['text'])

# Fill missing 'title' values with a placeholder
df['title'] = df['title'].fillna("No Title Provided")

# If there is an 'author' column, fill missing values with "Unknown"
if 'author' in df.columns:
    df['author'] = df['author'].fillna("Unknown")

In [5]:
# 5. Create the 'content' Column
# Combine 'title' and 'text' into a single text field
if 'content' not in df.columns:
    df['content'] = df['title'] + " " + df['text']

In [6]:
# 6. Split into Training and Testing Sets
X = df['content']
y = df['label']

# Split the data (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# 7. Preprocess Text for the LSTM (Updated)
def tokenize(text):
    # Convert to lowercase and split by word boundaries
    text = text.lower()
    tokens = re.findall(r'\b\w+\b', text)
    return tokens

# Build vocabulary from training texts, limiting to the top 10,000 words
all_tokens = []
for text in X_train:
    all_tokens.extend(tokenize(text))

freq = Counter(all_tokens)
max_vocab = 10000  # Limit vocabulary size to 10,000 words
vocab = {word: i+2 for i, (word, count) in enumerate(freq.most_common(max_vocab))}
vocab_size = len(vocab) + 2  # Reserve indices 0 for padding and 1 for unknown tokens

def text_to_sequence(text, vocab):
    tokens = tokenize(text)
    return [vocab.get(token, 1) for token in tokens]  # 1 for unknown words

# Convert texts to sequences of token indices
X_train_seq = [text_to_sequence(text, vocab) for text in X_train]
X_test_seq = [text_to_sequence(text, vocab) for text in X_test]

# Pad sequences to a fixed length (reduce max_len to lessen memory load)
max_len = 300  # Reduced from 500
def pad_sequence(seq, max_len):
    if len(seq) < max_len:
        return seq + [0] * (max_len - len(seq))
    else:
        return seq[:max_len]

X_train_pad = [pad_sequence(seq, max_len) for seq in X_train_seq]
X_test_pad = [pad_sequence(seq, max_len) for seq in X_test_seq]

# Convert padded sequences and labels to torch tensors
X_train_tensor = torch.tensor(X_train_pad, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_pad, dtype=torch.long)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

In [8]:
# 8. Create a PyTorch Dataset and DataLoader (Updated)
class NewsDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

batch_size = 16  # Reduced batch size to lower memory usage
train_dataset = NewsDataset(X_train_tensor, y_train_tensor)
test_dataset = NewsDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [9]:
# 8.1 Create a validation split from the training dataset

# Assume train_dataset is already defined (from your original Cell 8)
# Split train_dataset into training and validation subsets (e.g., 80% train, 20% validation)
from torch.utils.data import random_split

total_train = len(train_dataset)
val_size = int(0.2 * total_train)
train_size = total_train - val_size

train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Create DataLoader for the validation set
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

print(f"New train set size: {len(train_dataset)}, Validation set size: {len(val_dataset)}")

New train set size: 42022, Validation set size: 10505


In [10]:
def augment_sequence(seq, drop_prob=0.1):
    # For each token in the sequence (except padding tokens, which are 0),
    # with probability drop_prob, replace the token with 1 (the unknown token).
    augmented = [token if token == 0 or np.random.rand() > drop_prob else 1 for token in seq]
    return augmented

# Update your NewsDataset to support augmentation.
class NewsDataset(Dataset):
    def __init__(self, texts, labels, augment=False, drop_prob=0.1):
        self.texts = texts
        self.labels = labels
        self.augment = augment
        self.drop_prob = drop_prob
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        # Get the text tensor for the given index.
        text = self.texts[idx]
        label = self.labels[idx]
        if self.augment:
            # Convert tensor to list, augment, and convert back.
            text_list = text.tolist()
            text_list = augment_sequence(text_list, drop_prob=self.drop_prob)
            text = torch.tensor(text_list, dtype=torch.long)
        return text, label

# When creating your training dataset, enable augmentation:
train_dataset = NewsDataset(X_train_tensor, y_train_tensor, augment=True, drop_prob=0.1)
# For validation and test sets, do not augment:
val_dataset = NewsDataset(X_test_tensor, y_test_tensor, augment=False)
# (If you already have a validation split from Cell 8.1, use that dataset for validation without augmentation.)

In [11]:
# Cell 9: Pretrain the LSTM Autoencoder

class LSTMAutoencoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, latent_dim):
        super(LSTMAutoencoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.encoder = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc_enc = nn.Linear(hidden_dim, latent_dim)
        self.fc_dec = nn.Linear(latent_dim, embedding_dim)
        self.decoder = nn.LSTM(embedding_dim, embedding_dim, batch_first=True)
        self.output_layer = nn.Linear(embedding_dim, vocab_size)
        
    def forward(self, x):
        embedded = self.embedding(x)  # [batch, seq_length, embedding_dim]
        _, (hidden, _) = self.encoder(embedded)  # hidden: [num_layers, batch, hidden_dim]
        latent = self.fc_enc(hidden[-1])  # [batch, latent_dim]
        hidden_dec = self.fc_dec(latent).unsqueeze(0)  # [1, batch, embedding_dim]
        cell_dec = torch.zeros_like(hidden_dec)
        dec_out, _ = self.decoder(embedded, (hidden_dec, cell_dec))
        reconstructed = self.output_layer(dec_out)  # [batch, seq_length, vocab_size]
        return latent, reconstructed

# Hyperparameters for pretraining
pretrain_epochs = 10
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
embedding_dim = 100
hidden_dim = 128
latent_dim = 64

autoencoder = LSTMAutoencoder(vocab_size, embedding_dim, hidden_dim, latent_dim)
autoencoder.to(device)

criterion_recon = nn.CrossEntropyLoss(ignore_index=0)
optimizer_ae = optim.Adam(autoencoder.parameters(), lr=0.001)

best_recon_loss = float('inf')
for epoch in range(pretrain_epochs):
    autoencoder.train()
    running_loss = 0.0
    for texts, _ in train_loader:  # Using only texts; labels are not used here.
        texts = texts.to(device)
        optimizer_ae.zero_grad()
        _, reconstructed = autoencoder(texts)
        loss = criterion_recon(reconstructed.view(-1, vocab_size), texts.view(-1))
        loss.backward()
        optimizer_ae.step()
        running_loss += loss.item() * texts.size(0)
    avg_loss = running_loss / len(train_dataset)
    print(f"Autoencoder Pretrain Epoch {epoch+1}/{pretrain_epochs} - Loss: {avg_loss:.4f}")
    if avg_loss < best_recon_loss:
        best_recon_loss = avg_loss
        best_autoencoder_state = autoencoder.state_dict()

# Optionally load the best state
autoencoder.load_state_dict(best_autoencoder_state)

Autoencoder Pretrain Epoch 1/10 - Loss: 0.6030
Autoencoder Pretrain Epoch 2/10 - Loss: 0.0020
Autoencoder Pretrain Epoch 3/10 - Loss: 0.0003
Autoencoder Pretrain Epoch 4/10 - Loss: 0.0001
Autoencoder Pretrain Epoch 5/10 - Loss: 0.0000
Autoencoder Pretrain Epoch 6/10 - Loss: 0.0000
Autoencoder Pretrain Epoch 7/10 - Loss: 0.0000
Autoencoder Pretrain Epoch 8/10 - Loss: 0.0000
Autoencoder Pretrain Epoch 9/10 - Loss: 0.0000
Autoencoder Pretrain Epoch 10/10 - Loss: 0.0000


<All keys matched successfully>

In [12]:
# Cell 10: Define the Joint LSTM Autoencoder-Classifier Model with Increased Latent Dimension and Dropout
class JointLSTMAutoencoderClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, latent_dim, output_dim):
        super(JointLSTMAutoencoderClassifier, self).__init__()
        # Shared embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        
        # Encoder: LSTM to encode the input into a latent representation
        self.encoder = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc_enc = nn.Linear(hidden_dim, latent_dim)
        
        # Decoder: reconstruct the input from the latent vector
        self.fc_dec = nn.Linear(latent_dim, embedding_dim)
        self.decoder = nn.LSTM(embedding_dim, embedding_dim, batch_first=True)
        self.output_layer = nn.Linear(embedding_dim, vocab_size)
        
        # Classifier head with dropout regularization
        self.classifier = nn.Sequential(
            nn.Linear(latent_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.5),  # Added dropout here
            nn.Linear(64, output_dim)
        )
        
    def forward(self, x):
        # x shape: [batch_size, seq_length]
        embedded = self.embedding(x)  # [batch, seq_length, embedding_dim]
        _, (hidden, _) = self.encoder(embedded)  # hidden: [num_layers, batch, hidden_dim]
        latent = self.fc_enc(hidden[-1])         # [batch, latent_dim]
        
        # Classification output from latent representation
        class_output = self.classifier(latent)     # [batch, output_dim]
        
        # Reconstruction branch
        hidden_dec = self.fc_dec(latent).unsqueeze(0)  # [1, batch, embedding_dim]
        cell_dec = torch.zeros_like(hidden_dec)
        dec_out, _ = self.decoder(embedded, (hidden_dec, cell_dec))
        reconstructed = self.output_layer(dec_out)     # [batch, seq_length, vocab_size]
        
        return latent, reconstructed, class_output

# Increase latent dimension from 64 to 128
latent_dim = 128

# (Ensure that vocab_size, embedding_dim, hidden_dim, and output_dim have been defined.)

In [ ]:
# Cell 11: Two‑Phase Joint Training with Validation, Early Stopping, and Alpha Scheduling

# Set device and hyperparameters (make sure these are defined earlier if needed)
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
embedding_dim = 100   # as before
hidden_dim = 128      # as before
# latent_dim is now 128 (set above)
output_dim = 2        # binary classification

# Instantiate the joint model
joint_model = JointLSTMAutoencoderClassifier(vocab_size, embedding_dim, hidden_dim, latent_dim, output_dim)
joint_model.to(device)

# For this experiment, we will not load a pretrained autoencoder from an extra file.
# Instead, we train the joint model from scratch in two phases.

# Phase 1: Freeze the encoder (and optionally the embedding layer) and train only the classifier head.
for param in joint_model.encoder.parameters():
    param.requires_grad = False
joint_model.embedding.weight.requires_grad = False  # Freeze embeddings too

criterion_recon = nn.CrossEntropyLoss(ignore_index=0)  # Reconstruction loss
criterion_cls = nn.CrossEntropyLoss()                  # Classification loss
optimizer_joint = optim.Adam(joint_model.parameters(), lr=0.001)

patience = 3
best_val_acc = 0.0
patience_counter = 0

phase1_epochs = 5  # Train classifier head with frozen encoder
phase2_epochs = 5  # Fine-tune the entire network
initial_alpha = 0.5  # Starting weight for classification loss

# Lists to store metrics for plotting
train_loss_list = []
val_loss_list = []
train_acc_list = []
val_acc_list = []

total_epochs = phase1_epochs + phase2_epochs

for epoch in range(total_epochs):
    # In Phase 2, unfreeze the encoder and embeddings
    if epoch == phase1_epochs:
        print("Unfreezing encoder and embedding layer for Phase 2...")
        for param in joint_model.encoder.parameters():
            param.requires_grad = True
        joint_model.embedding.weight.requires_grad = True
    
    # Adjust alpha with a decay schedule (example: 10% decay per epoch)
    current_alpha = initial_alpha * (0.9 ** epoch)
    
    joint_model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    
    for texts, labels in train_loader:
        texts = texts.to(device)
        labels = labels.to(device)
        
        optimizer_joint.zero_grad()
        latent, reconstructed, class_output = joint_model(texts)
        
        loss_recon = criterion_recon(reconstructed.view(-1, vocab_size), texts.view(-1))
        loss_cls = criterion_cls(class_output, labels)
        loss = loss_recon + current_alpha * loss_cls
        
        loss.backward()
        optimizer_joint.step()
        
        running_loss += loss.item() * texts.size(0)
        _, predicted = torch.max(class_output, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()
    
    train_loss = running_loss / total_train
    train_acc = correct_train / total_train
    train_loss_list.append(train_loss)
    train_acc_list.append(train_acc)
    print(f"Epoch {epoch+1}/{total_epochs} - Train Loss: {train_loss:.4f} - Train Acc: {train_acc:.4f} - Alpha: {current_alpha:.4f}")
    
    # Evaluate on validation set (using val_loader from Cell 8.1)
    joint_model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for texts, labels in val_loader:
            texts = texts.to(device)
            labels = labels.to(device)
            _, _, class_output = joint_model(texts)
            loss_val = criterion_cls(class_output, labels)
            val_loss += loss_val.item() * texts.size(0)
            _, predicted_val = torch.max(class_output, 1)
            total_val += labels.size(0)
            correct_val += (predicted_val == labels).sum().item()
    
    avg_val_loss = val_loss / total_val
    val_acc = correct_val / total_val
    val_loss_list.append(avg_val_loss)
    val_acc_list.append(val_acc)
    print(f"Validation Loss: {avg_val_loss:.4f} - Validation Acc: {val_acc:.4f}")
    
    # Early stopping check
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        best_model_state = joint_model.state_dict()
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break

Epoch 1/10 - Train Loss: 1.0218 - Train Acc: 0.5556 - Alpha: 0.5000
Validation Loss: 0.6796 - Validation Acc: 0.5772


In [ ]:
# Cell 12: Plot Metrics and Evaluate on Test Set

import matplotlib.pyplot as plt

epochs_range = range(1, len(train_loss_list) + 1)

plt.figure(figsize=(12, 5))

# Plot Loss
plt.subplot(1, 2, 1)
plt.plot(epochs_range, train_loss_list, label='Train Loss')
plt.plot(epochs_range, val_loss_list, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss over Epochs')
plt.legend()

# Plot Accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs_range, train_acc_list, label='Train Accuracy')
plt.plot(epochs_range, val_acc_list, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy over Epochs')
plt.legend()

plt.tight_layout()
plt.show()

# Evaluate on test set
joint_model.eval()
correct_test = 0
total_test = 0

with torch.no_grad():
    for texts, labels in test_loader:
        texts = texts.to(device)
        labels = labels.to(device)
        _, _, class_output = joint_model(texts)
        _, predicted = torch.max(class_output, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

test_accuracy = correct_test / total_test
print("Joint Model Test Accuracy:", test_accuracy)

In [ ]:
# Cell 13: Plot Metrics and Evaluate on Test Set

import matplotlib.pyplot as plt

epochs_range = range(1, len(train_loss_list) + 1)

plt.figure(figsize=(12, 5))

# Plot Loss
plt.subplot(1, 2, 1)
plt.plot(epochs_range, train_loss_list, label='Train Loss')
plt.plot(epochs_range, val_loss_list, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss over Epochs')
plt.legend()

# Plot Accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs_range, train_acc_list, label='Train Accuracy')
plt.plot(epochs_range, val_acc_list, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy over Epochs')
plt.legend()

plt.tight_layout()
plt.show()

# Evaluate on test set
joint_model.eval()
correct_test = 0
total_test = 0

with torch.no_grad():
    for texts, labels in test_loader:
        texts = texts.to(device)
        labels = labels.to(device)
        _, _, class_output = joint_model(texts)
        _, predicted = torch.max(class_output, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

test_accuracy = correct_test / total_test
print("Joint Model Test Accuracy:", test_accuracy)